In [1]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [103]:
import datasets

train_df = datasets.load_from_disk("data/train")
test_df = datasets.load_from_disk("data/test")
validate_df = datasets.load_from_disk("data/validate")

In [22]:
item = train_df[11]
import pandas as pd

def to_pandas(item):
  return pd.DataFrame(item['table']["rows"],columns=item['table']["header"])

df = to_pandas(item)

In [80]:
client = Groq(api_key = your_api_key)


def normalize_column_name(name):
    return name.replace(" ", "").lower()

def decompose_table(example):
    table = to_pandas(example)
    table_markdown = table.to_markdown()
    query = example['query']
    title = example['table']['title']
    
    content_text = f"""Table Title:
{title}

Table:
{table_markdown}

User Query:
{query}

Please select the relevant columns from this table to answer the user's query.
"""
    
    completion = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful, respectful, honest, and intelligent assistant. Your task is to select relevant columns from a given table based on a user's natural language query. Follow these guidelines:\n1. Analyze the table structure, column headers, and all table data carefully.\n2. Interpret the user's query to understand their information need.\n3. Identify and select only the columns that are relevant to answering the user's query.\n4. If the query involves summarizing information, select all columns necessary for providing a comprehensive summary.\n5. If you cannot determine the relevant columns due to ambiguity or lack of clarity in the query, select all the columns to ensure no relevant data is omitted.\n6. Provide a list of the relevant column names. Do not include any other information in the output.\n7. Be precise and avoid selecting irrelevant columns."
            },
            {
                "role": "user",
                "content": content_text
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    full_response = ""

    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            full_response += chunk.choices[0].delta.content.strip()

    # Parse and normalize the response to get the list of relevant columns
    relevant_columns = [col.strip() for col in full_response.split(',') if col.strip()]
    relevant_columns_normalized = [normalize_column_name(col) for col in relevant_columns]

    # Normalize the table columns
    table_columns_normalized = {normalize_column_name(col): col for col in table.columns}

    # Map normalized column names back to original column names
    selected_columns = [table_columns_normalized[col] for col in relevant_columns_normalized if col in table_columns_normalized]

    # Check if the number of relevant columns is less than 3
    if len(selected_columns) < 3:
        selected_columns = table.columns.tolist()  # Keep all columns

    # Create the decomposed table
    decomposed_table = table[selected_columns]
    return decomposed_table

# List to store decomposed tables
decomposed_tables = []

# Process all examples and store decomposed tables
for i in tqdm(range(len(train_df))):
    example = train_df[i]
    decomposed_table = decompose_table(example)
    decomposed_tables.append(decomposed_table)

    

 24%|██▍       | 484/2000 [10:49<33:54,  1.34s/it]   


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j4yxxcd5fck8fy04nnbwfjas` on : Limit 1000000, Used 1000073, Requested 797. Please try again in 1m15.1788s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [113]:
decomposed_tables_test = []

In [121]:
decomposed_tables_validate = []

In [124]:
for i in tqdm(range(131, len(validate_df))):
    example = validate_df[i]
    decomposed_table = decompose_table(example)
    decomposed_tables_validate.append(decomposed_table)
        

100%|██████████| 69/69 [01:13<00:00,  1.07s/it]


In [97]:
for i in range(10):
    train_df[i]['table']['header'] = decomposed_tables[i].columns.tolist()
    train_df[i]['table']['rows'] = decomposed_tables[i].values.tolist()

# Save the modified dataset back to disk
train_df.save_to_disk("data/decomposed_train")

# Print a sample to verify
print(train_df[0])

Saving the dataset (1/1 shards): 100%|██████████| 2000/2000 [00:00<00:00, 3483.45 examples/s]

{'row_ids': [4], 'summary': "Tim Reeves and Dipash Chauhan had a mixed performance in the 2010 Isle Of Man TT. On the 29th of May, their race was cancelled, leaving them without a time. On the 31st, there wasn't a time for them. On the 1st, they recorded a time of 20'59.60, good for 107.834 mph. The following day, they posted a time of 20'45.81, a slight improvement at 109.028 mph. On the 3rd, they again improved their time to 20'26.35 and 110.758 mph. On the 4th, however, their time regressed to 37'03.92, a much slower 61.076 mph. Overall, it was a performance of ups and downs, with a few improvements from the start of the race to the end.", 'query': 'Summarize the performance of Tim Reeves and Dipash Chauhan in 2010 Isle Of Man TT.', 'table': {'header': ['Rank', 'Rider', 'Sat 29 May', 'Mon 31 May', 'Tues 1 June', 'Wed 2 June', 'Thurs 3 June', 'Fri 4 June'], 'rows': [['2', 'Klaus Klaffenböck / Dan Sayle 600Cc Lcr Honda', 'Cancelled No Time', "20'15.35 111.761 Mph", "20'05.79 112.647 M

In [126]:
def dataframe_to_dict(df):
    return {
        'header': list(df.columns),
        'rows': df.values.tolist()
    }

# Convert all DataFrames to dictionary format
decomposed_dicts = [dataframe_to_dict(df) for df in decomposed_tables_validate]

# Function to replace the table with the decomposed one
def replace_table(example, idx):
    example['table'] = decomposed_dicts[idx]
    return example

# Apply the replacement
decomposed_validate = validate_df.map(replace_table, with_indices=True)

Map: 100%|██████████| 200/200 [00:00<00:00, 646.95 examples/s]


In [129]:
decomposed_test.save_to_disk("data/decomposed_test")

Saving the dataset (1/1 shards): 100%|██████████| 500/500 [00:01<00:00, 441.63 examples/s]


In [19]:
from groq import Groq

completion = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful, respectful, honest and intelligent assistant. Your task is to select relevant columns from a given table based on a user's natural language query. Follow these guidelines:\n1. Analyze the table structure and column headers carefully.\n2. Interpret the user's query to understand their information need.\n3. Identify and select only the columns that are relevant to answering the user's query.\n4. If the query involves summarizing information, select all columns necessary for providing a comprehensive summary.\n5. If you cannot determine the relevant columns due to ambiguity or lack of clarity in the query, select all the columns to ensure no relevant data is omitted.\n6. Provide a list of the relevant column names. Do not include any other information in the output\n7. Be precise and avoid selecting irrelevant columns.."
        },
        {
            "role": "user",
            "content": "Table:\n| Product Name | Category | Price | Stock | Manufacturer | Color | Weight (kg) | Release Date |\n\nUser Query: \"What are the available blue products under $50?\"\n\nPlease select the relevant columns from this table to answer the user's query."
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


Product Name, 
Color, 
Price